In [ ]:
import pandas as pd
import numpy as np 

# Get the number of each priority

In [ ]:
def get_prio_total(filename):
    '''read the data trace and get the number of block in each priority'''
    prio_total = {}
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            line_soup = line.split()
            if prio_total.get(line_soup[3]) is None:
                prio_total[line_soup[3]] = 1
            else:
                prio_total[line_soup[3]] += 1
    print(prio_total)
    return prio_total

In [ ]:
def calc_throughput(filepath):
    with open(filepath, "r") as f:
        client_lines = f.readlines()
        client_stats = client_lines[-1].split(",")
        total_time = int(client_stats[-1]) / 1000000 - 3 # s
        total_bytes = int(client_stats[-4]) # B
        return total_bytes * 8 / total_time / 1000000 # Mbps

In [ ]:
def parse_file(filename):
    cols = {}
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            words = lines[i].split()
            if len(words) == 5:
                if words[0] == 'BlockID':
                    for word in words:
                        cols[word] = []
                    continue
                else:
                    cols['BlockID'].append(int(words[0]))
                    cols['bct'].append(int(words[1]))
                    cols['BlockSize'].append(int(words[2]))
                    cols['Priority'].append(int(words[3]))
                    cols['Deadline'].append(int(words[4]))
            else:
                continue
    return pd.DataFrame(cols)

In [ ]:
def parse_client_csv(filepath):
    client_df = pd.read_csv(filepath)
    return client_df

def get_bct_stat(client_df, prio_total):
    avg_bct = sum(client_df["bct"]) / len(client_df["bct"])
    avg_prio_bct = {}
    priok_bct_ddl = None
    prio_bct_complete_rate = {}
    for prio in prio_total.keys():
        if len(client_df["bct"][client_df["Priority"] == int(prio)]) <= 0:
            continue
        avg_prio_bct[prio] = sum(client_df["bct"][client_df["Priority"] == int(prio)]) / len(client_df["bct"][client_df["Priority"] == int(prio)])
        priok_bct_ddl = client_df[["bct", "Deadline"]][client_df["Priority"] == int(prio)]

        priok_before_ddl = 0
        for i in range(len(priok_bct_ddl)):
            if priok_bct_ddl.iloc[i]["bct"] < priok_bct_ddl.iloc[i]["Deadline"]:
                priok_before_ddl += 1
        prio_bct_complete_rate[prio] = priok_before_ddl / prio_total[prio]

    print("avg_bct", avg_bct)
    print("avg_prio_bct", avg_prio_bct)
    print("prio_complete_rate", prio_bct_complete_rate)
    return (avg_bct, avg_prio_bct, prio_bct_complete_rate)

In [ ]:
import os
# prio_total = get_prio_total("examples/dtp_test/trace/block_trace/aitrans_block.txt")
prio_total = get_prio_total("/home/ubuntu/dtp_test_scripts/dataset/converted_data/GTA.csv")
for root, dirs, files in os.walk("examples/dtp_test/logs"):
    if not root.endswith("logs"):
        print("******",root, "******")
        for filename in files:
            if filename.find("client-") >= 0:
                print("======", filename, "======================")
                client_df = parse_file(os.path.join(root, filename))
                get_bct_stat(client_df, prio_total)

In [ ]:
_ = get_prio_total("/home/ubuntu/dtp_test_scripts/dataset/converted_data/GTA.csv")